In [364]:
import pandas as pd
import mysql.connector
import os
import numpy as np
import warnings
from sklearn.metrics import classification_report

warnings.filterwarnings('ignore')


# Datos sucios

In [365]:
patients_batch1 = pd.read_csv("../data/patients_batch1.csv")
patients_batch1

,ehr,birth_date,diagnosis_date,death_date
0,10011773,1959-07-05,2015-04-02,NaN
1,10020495,1953-10-02,2017-12-04,NaN
2,10030299,1966-08-16,2019-06-27,NaN
3,10030824,1953-03-03,2018-09-07,NaN
4,10041592,1959-07-06,2018-11-19,NaN
...,...,...,...,...
7495,77575810,1958-03-21,2014-02-22,NaN
7496,77578551,1948-05-18,2018-07-11,NaN
7497,77579212,1976-02-13,2022-03-22,NaN
7498,77605742,1950-03-16,2015-06-29,NaN


In [366]:
patients_batch2 = pd.read_csv("../data/patients_batch2.csv")
patients_batch2

,ehr,birth_date,diagnosis_date,death_date
0,72992494,13/02/1942,08/05/2018,NaN
1,73002338,22/10/1949,08/09/2018,NaN
2,73008149,04/12/1969,12/04/2018,NaN
3,73012939,18/09/1973,17/04/2014,01/01/2018
4,73019870,07/12/1955,24/05/2018,NaN
...,...,...,...,...
2995,99966287,16/12/1978,02/08/2018,NaN
2996,99981192,10/04/1949,28/10/2017,NaN
2997,99988958,21/01/1966,27/05/2013,NaN
2998,99992350,04/09/1939,17/12/2018,NaN


In [367]:
gynecological = pd.read_excel("../data/gynecological.xlsx")
gynecological

,Unnamed: 0,ehr,pregnancy,birth,caesarean,abort,menarche_age,menopause_age
0,0,10011773,-8.0,0.0,NaN,0.0,NaN,71.0
1,1,10030299,3.0,NaN,0.0,0.0,NaN,NaN
2,2,10030824,0.0,0.0,0.0,NaN,NaN,44.0
3,3,10053435,2.0,NaN,0.0,1.0,21.0,74.0
4,4,10111454,0.0,-6.0,NaN,0.0,16.0,55.0
...,...,...,...,...,...,...,...,...
7681,7681,99948591,9.0,2.0,-4.0,0.0,12.0,47.0
7682,7682,99961100,2.0,2.0,0.0,0.0,19.0,52.0
7683,7683,99981192,3.0,3.0,0.0,-5.0,11.0,NaN
7684,7684,99992350,4.0,3.0,NaN,-6.0,11.0,55.0


In [368]:
histochemistry = pd.read_json("../data/histochemistry.json")
histochemistry

,ehr,er,her2,ki67,pr
0,10011773,0.0,0.0,19.0,NaN
1,10020495,1.0,0.0,9.0,0.0
2,10030299,1.0,NaN,18.0,1.0
3,10030824,1.0,0.0,NaN,1.0
4,10041592,0.0,0.0,65.0,0.0
...,...,...,...,...,...
9995,99966287,1.0,0.0,NaN,0.0
9996,99981192,1.0,0.0,14.0,0.0
9997,99988958,NaN,0.0,67.0,NaN
9998,99992350,0.0,0.0,16.0,0.0


In [369]:
if os.path.exists('../data/tumor.xlsx'):
    tumor = pd.read_excel("../data/tumor.xlsx")
else:
    config = {'host': "localhost",
              'port': 3306,
              'user': "cancer_mama_user",
              'passwd': "cancer_mama_password",
              'db': "cancer_mama"}
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()

    sql_columns_names = "SHOW COLUMNS from tumor;"
    sql_tumor = "SELECT * FROM tumor"

    cursor.execute(sql_tumor)
    tumor = pd.DataFrame(cursor.fetchall())

    cursor.execute(sql_columns_names)
    columns_names = pd.DataFrame(cursor.fetchall())

    cursor.close()
    conn.close()

    tumor = tumor.set_axis(columns_names[0], axis=1)
    tumor.to_excel("../data/tumor.xlsx")
tumor

,Unnamed: 0,ehr,n_tumor,t_category,n_category,m_category,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_diagnosis,stage_after_neo,grade,ductal,lobular,neoadjuvant
0,0,10011773,1,IS,0,0,NaN,NaN,NaN,0,NaN,1,NaN,NaN,no
1,1,10020495,1,1,0,0,NaN,NaN,NaN,IA,NaN,2,1.0,NaN,no
2,2,10020495,2,3,1,0,2,0.0,NaN,IA,NaN,2,1.0,NaN,yes
3,3,10030299,1,1,0,0,NaN,NaN,NaN,IA,NaN,1,1.0,NaN,no
4,4,10030824,1,2,1,0,2,2.0,0.0,IIIA,IIIA,3,NaN,1.0,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11162,11162,99966287,1,1,1,0,1,0.0,0.0,IB,IIA,3,1.0,NaN,yes
11163,11163,99981192,1,1,0,0,NaN,NaN,NaN,IA,NaN,2,NaN,NaN,no
11164,11164,99988958,1,0,2,0,1,1.0,0.0,IIIA,IIA,1,1.0,NaN,yes
11165,11165,99992350,1,2,0,0,3,2.0,0.0,IIA,IIIA,2,1.0,NaN,yes


# Preprocesamiento

## Patients

In [370]:
len(patients_batch1.ehr.unique())

7500

In [371]:
len(patients_batch2.ehr.unique())

3000

Vemos si hay algún paciente común en ambos batchs de datos de pacientes

In [372]:
any(patients_batch1['ehr'].isin(patients_batch2['ehr']))

True

Obtenemos los ids de los pacientes que aparecen en ambos 

In [373]:
elementos_comunes = set(patients_batch1['ehr']).intersection(set(patients_batch2['ehr']))
print(f"El número total de pacientes comunes son: {len(elementos_comunes)}")

El número total de pacientes comunes son: 500


Vemos ahora si estos pacientes que aparecen en ambos documentos tienen valores distintos en cada uno de los documentos

In [374]:
pacientes_comunes_batch1 = patients_batch1[patients_batch1['ehr'].isin(elementos_comunes)]
pacientes_comunes_batch1

,ehr,birth_date,diagnosis_date,death_date
7000,72992494,1942-02-13,2018-05-08,NaN
7001,73002338,1949-10-22,2018-09-08,NaN
7002,73008149,1969-12-04,2018-04-12,NaN
7003,73012939,1973-09-18,2014-04-17,2018-01-01
7004,73019870,1955-12-07,2018-05-24,NaN
...,...,...,...,...
7495,77575810,1958-03-21,2014-02-22,NaN
7496,77578551,1948-05-18,2018-07-11,NaN
7497,77579212,1976-02-13,2022-03-22,NaN
7498,77605742,1950-03-16,2015-06-29,NaN


In [375]:
pacientes_comunes_batch2 = patients_batch2[patients_batch2['ehr'].isin(elementos_comunes)]
pacientes_comunes_batch2

,ehr,birth_date,diagnosis_date,death_date
0,72992494,13/02/1942,08/05/2018,NaN
1,73002338,22/10/1949,08/09/2018,NaN
2,73008149,04/12/1969,12/04/2018,NaN
3,73012939,18/09/1973,17/04/2014,01/01/2018
4,73019870,07/12/1955,24/05/2018,NaN
...,...,...,...,...
495,77575810,21/03/1958,22/02/2014,NaN
496,77578551,18/05/1948,11/07/2018,NaN
497,77579212,13/02/1976,22/03/2022,NaN
498,77605742,16/03/1950,29/06/2015,NaN


Parece ser que los valores sí que son iguales, pero cada documento usa un formato distinto para las fechas. Vamos a ponerlo todo en el formato del primer documento, que es el formato por defecto de datetime: aaaa-mm-dd

In [376]:
pacientes_comunes_batch2['birth_date'] = pd.to_datetime(pacientes_comunes_batch2['birth_date'], format='%d/%m/%Y')
pacientes_comunes_batch2['death_date'] = pd.to_datetime(pacientes_comunes_batch2['death_date'], format='%d/%m/%Y')
pacientes_comunes_batch2['diagnosis_date'] = pd.to_datetime(pacientes_comunes_batch2['diagnosis_date'], format='%d/%m/%Y')

pacientes_comunes_batch1['birth_date'] = pd.to_datetime(pacientes_comunes_batch1['birth_date'])
pacientes_comunes_batch1['diagnosis_date'] = pd.to_datetime(pacientes_comunes_batch1['diagnosis_date'])
pacientes_comunes_batch1['death_date'] = pd.to_datetime(pacientes_comunes_batch1['death_date'])

pacientes_comunes_batch2

,ehr,birth_date,diagnosis_date,death_date
0,72992494,1942-02-13,2018-05-08,NaT
1,73002338,1949-10-22,2018-09-08,NaT
2,73008149,1969-12-04,2018-04-12,NaT
3,73012939,1973-09-18,2014-04-17,2018-01-01
4,73019870,1955-12-07,2018-05-24,NaT
...,...,...,...,...
495,77575810,1958-03-21,2014-02-22,NaT
496,77578551,1948-05-18,2018-07-11,NaT
497,77579212,1976-02-13,2022-03-22,NaT
498,77605742,1950-03-16,2015-06-29,NaT


In [377]:
pacientes_comunes_batch1['birth_date'] = pd.to_datetime(pacientes_comunes_batch1['birth_date'])
pacientes_comunes_batch1['diagnosis_date'] = pd.to_datetime(pacientes_comunes_batch1['diagnosis_date'])
pacientes_comunes_batch1['death_date'] = pd.to_datetime(pacientes_comunes_batch1['death_date'])

In [378]:
pacientes_comunes_batch1.reset_index(drop=True, inplace=True)

In [379]:
pacientes_comunes_batch2.equals(pacientes_comunes_batch1)

True

Vemos que todos los valores de los pacientes que aparecen en los dos documentos son exactamente iguales

In [380]:
patients_batch1

,ehr,birth_date,diagnosis_date,death_date
0,10011773,1959-07-05,2015-04-02,NaN
1,10020495,1953-10-02,2017-12-04,NaN
2,10030299,1966-08-16,2019-06-27,NaN
3,10030824,1953-03-03,2018-09-07,NaN
4,10041592,1959-07-06,2018-11-19,NaN
...,...,...,...,...
7495,77575810,1958-03-21,2014-02-22,NaN
7496,77578551,1948-05-18,2018-07-11,NaN
7497,77579212,1976-02-13,2022-03-22,NaN
7498,77605742,1950-03-16,2015-06-29,NaN


In [381]:
patients_batch2

,ehr,birth_date,diagnosis_date,death_date
0,72992494,13/02/1942,08/05/2018,NaN
1,73002338,22/10/1949,08/09/2018,NaN
2,73008149,04/12/1969,12/04/2018,NaN
3,73012939,18/09/1973,17/04/2014,01/01/2018
4,73019870,07/12/1955,24/05/2018,NaN
...,...,...,...,...
2995,99966287,16/12/1978,02/08/2018,NaN
2996,99981192,10/04/1949,28/10/2017,NaN
2997,99988958,21/01/1966,27/05/2013,NaN
2998,99992350,04/09/1939,17/12/2018,NaN


In [382]:
patients_batch2['birth_date'] = pd.to_datetime(patients_batch2['birth_date'], format='%d/%m/%Y')
patients_batch2['diagnosis_date'] = pd.to_datetime(patients_batch2['diagnosis_date'], format='%d/%m/%Y')
patients_batch2['birth_date'] = pd.to_datetime(patients_batch2['birth_date'], format='%d/%m/%Y')

patients_batch1['birth_date'] = pd.to_datetime(patients_batch1['birth_date'])
patients_batch1['diagnosis_date'] = pd.to_datetime(patients_batch1['diagnosis_date'])
patients_batch1['death_date'] = pd.to_datetime(patients_batch1['death_date'])

In [383]:
patients_batch1_sin_comunes = patients_batch1[~patients_batch1["ehr"].isin(pacientes_comunes_batch1["ehr"])]
patients_batch1_sin_comunes

,ehr,birth_date,diagnosis_date,death_date
0,10011773,1959-07-05,2015-04-02,NaT
1,10020495,1953-10-02,2017-12-04,NaT
2,10030299,1966-08-16,2019-06-27,NaT
3,10030824,1953-03-03,2018-09-07,NaT
4,10041592,1959-07-06,2018-11-19,NaT
...,...,...,...,...
6995,72964107,1959-01-01,2019-12-18,NaT
6996,72975846,1959-07-02,2017-09-01,NaT
6997,72976171,1972-12-01,2018-06-09,NaT
6998,72983987,1962-08-28,2013-07-03,2019-01-25


In [384]:
patients = pd.concat([patients_batch1_sin_comunes, patients_batch2], ignore_index=True)
patients

,ehr,birth_date,diagnosis_date,death_date
0,10011773,1959-07-05,2015-04-02,NaN
1,10020495,1953-10-02,2017-12-04,NaN
2,10030299,1966-08-16,2019-06-27,NaN
3,10030824,1953-03-03,2018-09-07,NaN
4,10041592,1959-07-06,2018-11-19,NaN
...,...,...,...,...
9995,99966287,1978-12-16,2018-08-02,NaN
9996,99981192,1949-04-10,2017-10-28,NaN
9997,99988958,1966-01-21,2013-05-27,NaN
9998,99992350,1939-09-04,2018-12-17,NaN


In [385]:
len(patients.ehr.unique())

10000

Ahora deberemos sustituir las columnas de fechas por unas numericas. De esta forma 'birth_date' sera 'age' (edad actual del paciente), 'diagnosis_date' sera 'diagnosis_age' (edad a la que fue diagnsoticado).

In [386]:
from datetime import datetime
patients['age'] = datetime.now().year - patients['birth_date'].dt.year
patients['diagnosis_age'] = patients['diagnosis_date'].dt.year - patients['birth_date'].dt.year


Por ultimo, hay muchos valores nulos en la variable 'death_date'. Esto sinifica que la mayoria de los pacientes no han muerto. Prodedemos a cambiar esta variable por una binaria que indique tan solo si el paciente ha fallecido o no.

In [387]:
patients.isnull().sum()

ehr                  0
birth_date           0
diagnosis_date       0
death_date        8865
age                  0
diagnosis_age        0
dtype: int64

In [388]:
patients['dead'] = patients['death_date'].notnull().astype(int)
patients.drop(columns=['death_date', 'birth_date', 'diagnosis_date'], inplace=True)

In [389]:
patients

,ehr,age,diagnosis_age,dead
0,10011773,65,56,0
1,10020495,71,64,0
2,10030299,58,53,0
3,10030824,71,65,0
4,10041592,65,59,0
...,...,...,...,...
9995,99966287,46,40,0
9996,99981192,75,68,0
9997,99988958,58,47,0
9998,99992350,85,79,0


## Histochemistry

In [390]:
histochemistry

,ehr,er,her2,ki67,pr
0,10011773,0.0,0.0,19.0,NaN
1,10020495,1.0,0.0,9.0,0.0
2,10030299,1.0,NaN,18.0,1.0
3,10030824,1.0,0.0,NaN,1.0
4,10041592,0.0,0.0,65.0,0.0
...,...,...,...,...,...
9995,99966287,1.0,0.0,NaN,0.0
9996,99981192,1.0,0.0,14.0,0.0
9997,99988958,NaN,0.0,67.0,NaN
9998,99992350,0.0,0.0,16.0,0.0


In [391]:
histochemistry.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ehr     10000 non-null  int64  
 1   er      9042 non-null   float64
 2   her2    9006 non-null   float64
 3   ki67    9032 non-null   float64
 4   pr      9018 non-null   float64
dtypes: float64(4), int64(1)
memory usage: 390.8 KB


Todos los valores estan dentro de los rangos posibles salvo la variable 'ki67' que al ser un valor porcentual no podria sobrepasar el valor 100, y sin embargo vemos que su valor maximo es 137.

In [392]:
histochemistry.describe()

,ehr,er,her2,ki67,pr
count,1.000000e+04,9042.000000,9006.000000,9032.000000,9018.000000
mean,5.494471e+07,0.662906,0.158894,21.315766,0.665669
std,2.609028e+07,0.472743,0.365598,22.242693,0.471782
min,1.001177e+07,0.000000,0.000000,-57.000000,0.000000
25%,3.214794e+07,0.000000,0.000000,12.000000,0.000000
50%,5.480334e+07,1.000000,0.000000,18.000000,1.000000
75%,7.761323e+07,1.000000,0.000000,28.000000,1.000000
max,9.999442e+07,1.000000,1.000000,137.000000,1.000000


Procedemos a tratar como nulos aquellos valores que se encuentren fuera de rango.

In [393]:
print('Numero de pacientes con ki67 fuera de rango: ', len(histochemistry[histochemistry['ki67'] > 100] ))
histochemistry.loc[histochemistry['ki67'] > 100, 'ki67'] = pd.NA

Numero de pacientes con ki67 fuera de rango:  19


En todas la columnas nos encontramos con valores nulos.

In [394]:
histochemistry.isnull().sum()

ehr       0
er      958
her2    994
ki67    987
pr      982
dtype: int64

No hay ninguna fila completamente vacia

In [395]:
filas_nulas = histochemistry[histochemistry.columns.drop('ehr')].isnull().all(axis=1)
filas_nulas[filas_nulas].index.tolist()

[]

Sin embargo hay 489 filas con 2 o mas valores nulos. Procedemos a eliminarlas puesto que este df tiene tan solo 4 variables y si faltan la mitad o mas consideramos que no aporta informacion sufieciente. Ademas tan solo hay 489 filas a las que les sucede, que corresponden tan solo a un 5% del dataset. 

In [396]:
filas_nulas = histochemistry[histochemistry.columns.drop('ehr')].isnull().sum(axis=1) >=2
len(filas_nulas[filas_nulas].index.tolist())

489

In [397]:
histochemistry.drop(filas_nulas[filas_nulas].index.tolist(), inplace=True)

Sigue habiendo valores nulos, pero al menos ahora sabemos que en cada fila como mucho hay tan solo 1 valor nulo

In [398]:
histochemistry.isnull().sum()

ehr       0
er      698
her2    757
ki67    720
pr      725
dtype: int64

In [399]:
len(histochemistry.ehr.unique())

9511

Coinciden todos los ids con los de patients

In [400]:
elementos_comunes = set(histochemistry['ehr']).intersection(set(patients['ehr']))
len(elementos_comunes)

9511

Como comparten todos los ids simplemente añadimos la informacion del nuevo df al de 'patients'

In [401]:
patients = pd.merge(patients, histochemistry, on='ehr')

In [402]:
patients

,ehr,age,diagnosis_age,dead,er,her2,ki67,pr
0,10011773,65,56,0,0.0,0.0,19.0,NaN
1,10020495,71,64,0,1.0,0.0,9.0,0.0
2,10030299,58,53,0,1.0,NaN,18.0,1.0
3,10030824,71,65,0,1.0,0.0,NaN,1.0
4,10041592,65,59,0,0.0,0.0,65.0,0.0
...,...,...,...,...,...,...,...,...
9506,99963879,65,62,0,1.0,0.0,18.0,1.0
9507,99966287,46,40,0,1.0,0.0,NaN,0.0
9508,99981192,75,68,0,1.0,0.0,14.0,0.0
9509,99992350,85,79,0,0.0,0.0,16.0,0.0


## Gynecological

In [403]:
gynecological

,Unnamed: 0,ehr,pregnancy,birth,caesarean,abort,menarche_age,menopause_age
0,0,10011773,-8.0,0.0,NaN,0.0,NaN,71.0
1,1,10030299,3.0,NaN,0.0,0.0,NaN,NaN
2,2,10030824,0.0,0.0,0.0,NaN,NaN,44.0
3,3,10053435,2.0,NaN,0.0,1.0,21.0,74.0
4,4,10111454,0.0,-6.0,NaN,0.0,16.0,55.0
...,...,...,...,...,...,...,...,...
7681,7681,99948591,9.0,2.0,-4.0,0.0,12.0,47.0
7682,7682,99961100,2.0,2.0,0.0,0.0,19.0,52.0
7683,7683,99981192,3.0,3.0,0.0,-5.0,11.0,NaN
7684,7684,99992350,4.0,3.0,NaN,-6.0,11.0,55.0


Verificamos que no haya pacientes repetidos

In [404]:
len(gynecological.ehr.unique()) == len(gynecological)


True

In [405]:
gynecological.drop(columns=['Unnamed: 0'], inplace=True)

In [406]:
gynecological.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7686 entries, 0 to 7685
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ehr            7686 non-null   int64  
 1   pregnancy      6413 non-null   float64
 2   birth          6417 non-null   float64
 3   caesarean      6131 non-null   float64
 4   abort          6235 non-null   float64
 5   menarche_age   4901 non-null   float64
 6   menopause_age  4486 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 420.5 KB


Observamos como hay valores fuera de rango. Para las variables 'pregnancy', 'birth', 'caesarean', 'abort' los valores minimos son negativos, y los maximos dudamos que puedan ser reales. Para la variable 'menopause_age' el valor maximo es 145, lo cual es imposible. Habra que tratar estos valores asi como deberemos asegurarnos que el valor de pregnancy = birth + caesarean + abort.

In [407]:
gynecological.describe()

,ehr,pregnancy,birth,caesarean,abort,menarche_age,menopause_age
count,7.686000e+03,6413.000000,6417.000000,6131.000000,6235.000000,4901.000000,4486.000000
mean,5.505640e+07,2.078590,1.673991,0.029848,0.259984,12.736584,50.263041
std,2.621591e+07,3.052473,3.006228,2.677790,2.691482,3.062710,17.196300
min,1.001177e+07,-10.000000,-10.000000,-11.000000,-12.000000,0.000000,2.000000
25%,3.202165e+07,1.000000,0.000000,0.000000,0.000000,11.000000,44.000000
50%,5.503108e+07,2.000000,2.000000,0.000000,0.000000,13.000000,49.000000
75%,7.794185e+07,3.000000,3.000000,0.000000,1.000000,14.000000,53.000000
max,9.999442e+07,14.000000,15.000000,12.000000,12.000000,26.000000,145.000000


Vamos a estudiar qué hacer con los valores fuera de rango. Comenzamos con la clase 'pregnancy', la que tenemos 585 filas en las que hay un numero negativo de embarazos, lo cual es imposible. De momento estableceremos estos valores como nulos para su posterior tratamiento. 

In [408]:
print('Numero de pacientes con numero de embarazos fuera de rango: ', len(gynecological[gynecological['pregnancy'] < 0] ))
gynecological.loc[gynecological['pregnancy'] < 0, 'pregnancy'] = pd.NA

Numero de pacientes con numero de embarazos fuera de rango:  585


Analogamente para las variables 'birth', 'caesarean', 'abort'

In [409]:
print('Numero de pacientes con numero de partos fuera de rango: ', len(gynecological[gynecological['birth'] < 0] ))
gynecological.loc[gynecological['birth'] < 0, 'birth'] = pd.NA

print('Numero de pacientes con numero de cesareas fuera de rango: ', len(gynecological[gynecological['caesarean'] < 0] ))
gynecological.loc[gynecological['caesarean'] < 0, 'caesarean'] = pd.NA

print('Numero de pacientes con numero de abortos fuera de rango: ', len(gynecological[gynecological['abort'] < 0] ))
gynecological.loc[gynecological['abort'] < 0, 'abort'] = pd.NA


Numero de pacientes con numero de partos fuera de rango:  609
Numero de pacientes con numero de cesareas fuera de rango:  667
Numero de pacientes con numero de abortos fuera de rango:  617


Hay una gran cantidad de nulos, sobre todo para la variable 'menopause_age', siendo el 41.6% de los valores nulos. Teniendo esto en cuenta procederemos a no considerar esta variable. 

In [410]:
gynecological.isnull().sum()

ehr                 0
pregnancy        1858
birth            1878
caesarean        2222
abort            2068
menarche_age     2785
menopause_age    3200
dtype: int64

In [411]:
gynecological.drop(columns='menopause_age', inplace=True)

Una vez hemos eliminado valores fuera de rango consideraremos los valores, que a pesar de estar dentro dee rango, son atipicos

# FALTA IMPUTAR LOS EMBARAZOS Y DESPUES VER Q HACER CON OUTLIERS

LO DE LOS OUTLIERS EL CODIGO ESTA, SOLO FALTA EL DE IMPUTACION


In [412]:
Q1 = gynecological['pregnancy'].quantile(0.25)
Q3 = gynecological['pregnancy'].quantile(0.75)
IQR = Q3 - Q1

# Definir los límites para considerar un dato como atípico
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identificar los valores atípicos
outliers = gynecological[(gynecological['pregnancy'] < lower_bound) | (gynecological['pregnancy'] > upper_bound)]

#Eliminamos los pacientes con valores atipicos de embarazos
# gynecological = gynecological.drop(outliers.index, inplace=True)


Sin contar la variable identificadora 'ehr' tenemos tan solo 5 variables explicativas en este dataset. Porcedemos a eliminar aquellas filas que tengan tan solo 1 valor no nulo. Podriamos optar a eliminar aquellas con tan solo 2 valores no nulos, pero primero debremos estudiar si podemos imputar nosotros esos valores.

In [413]:
filas_nulas = gynecological[gynecological.columns.drop('ehr')].isnull().sum(axis=1) >=4
len(filas_nulas[filas_nulas].index.tolist())

135

In [414]:
gynecological.drop(filas_nulas[filas_nulas].index.tolist(), inplace=True)

In [415]:
len(gynecological.ehr.unique())


7551

In [416]:
elementos_comunes = set(gynecological['ehr']).intersection(set(patients['ehr']))
len(elementos_comunes)

7184

In [417]:
gynecological_patients = pd.merge(gynecological, patients, on='ehr')

In [418]:
gynecological_patients

,ehr,pregnancy,birth,caesarean,abort,menarche_age,age,diagnosis_age,dead,er,her2,ki67,pr
0,10011773,NaN,0.0,NaN,0.0,NaN,65,56,0,0.0,0.0,19.0,NaN
1,10030299,3.0,NaN,0.0,0.0,NaN,58,53,0,1.0,NaN,18.0,1.0
2,10030824,0.0,0.0,0.0,NaN,NaN,71,65,0,1.0,0.0,NaN,1.0
3,10053435,2.0,NaN,0.0,1.0,21.0,67,60,0,1.0,0.0,NaN,1.0
4,10111454,0.0,NaN,NaN,0.0,16.0,63,56,0,1.0,0.0,72.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7179,99948591,9.0,2.0,NaN,0.0,12.0,69,67,0,1.0,0.0,12.0,1.0
7180,99961100,2.0,2.0,0.0,0.0,19.0,54,48,0,1.0,0.0,21.0,1.0
7181,99981192,3.0,3.0,0.0,NaN,11.0,75,68,0,1.0,0.0,14.0,0.0
7182,99992350,4.0,3.0,NaN,NaN,11.0,85,79,0,0.0,0.0,16.0,0.0


## Tumor

In [419]:
tumor

,Unnamed: 0,ehr,n_tumor,t_category,n_category,m_category,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_diagnosis,stage_after_neo,grade,ductal,lobular,neoadjuvant
0,0,10011773,1,IS,0,0,NaN,NaN,NaN,0,NaN,1,NaN,NaN,no
1,1,10020495,1,1,0,0,NaN,NaN,NaN,IA,NaN,2,1.0,NaN,no
2,2,10020495,2,3,1,0,2,0.0,NaN,IA,NaN,2,1.0,NaN,yes
3,3,10030299,1,1,0,0,NaN,NaN,NaN,IA,NaN,1,1.0,NaN,no
4,4,10030824,1,2,1,0,2,2.0,0.0,IIIA,IIIA,3,NaN,1.0,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11162,11162,99966287,1,1,1,0,1,0.0,0.0,IB,IIA,3,1.0,NaN,yes
11163,11163,99981192,1,1,0,0,NaN,NaN,NaN,IA,NaN,2,NaN,NaN,no
11164,11164,99988958,1,0,2,0,1,1.0,0.0,IIIA,IIA,1,1.0,NaN,yes
11165,11165,99992350,1,2,0,0,3,2.0,0.0,IIA,IIIA,2,1.0,NaN,yes


In [420]:
tumor.drop(columns=['Unnamed: 0'], inplace=True)

hay ids que se repiten. una persona puede tener varios tumores

In [421]:
len(tumor.ehr.unique())

9971

In [422]:
elementos_comunes = set(tumor['ehr']).intersection(set(gynecological_patients['ehr']))
len(elementos_comunes)

7166

Tiene mas elementos en comun con 'patients' que con 'gynecological_patients'

In [423]:
elementos_comunes = set(tumor['ehr']).intersection(set(patients['ehr']))
len(elementos_comunes)

9484

In [424]:
tumor_patients = pd.merge(tumor, patients, on='ehr')

Finalmente obtenemos un df en el que se incluye la informacion de todos los pasados salvo 'gynecological'. Sin embargo en este df no hay una unica fila por cada paciente, sino que puede haber varias debido a que un paciente puede tener varios tumores, dando lugar a una duplicidad parcial de filas

In [425]:
tumor_patients

,ehr,n_tumor,t_category,n_category,m_category,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_diagnosis,stage_after_neo,...,ductal,lobular,neoadjuvant,age,diagnosis_age,dead,er,her2,ki67,pr
0,10011773,1,IS,0,0,NaN,NaN,NaN,0,NaN,...,NaN,NaN,no,65,56,0,0.0,0.0,19.0,NaN
1,10020495,1,1,0,0,NaN,NaN,NaN,IA,NaN,...,1.0,NaN,no,71,64,0,1.0,0.0,9.0,0.0
2,10020495,2,3,1,0,2,0.0,NaN,IA,NaN,...,1.0,NaN,yes,71,64,0,1.0,0.0,9.0,0.0
3,10030299,1,1,0,0,NaN,NaN,NaN,IA,NaN,...,1.0,NaN,no,58,53,0,1.0,NaN,18.0,1.0
4,10030824,1,2,1,0,2,2.0,0.0,IIIA,IIIA,...,NaN,1.0,yes,71,65,0,1.0,0.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10620,99963879,1,2,3,0,3,1.0,0.0,IIIC,0,...,1.0,NaN,yes,65,62,0,1.0,0.0,18.0,1.0
10621,99966287,1,1,1,0,1,0.0,0.0,IB,IIA,...,1.0,NaN,yes,46,40,0,1.0,0.0,NaN,0.0
10622,99981192,1,1,0,0,NaN,NaN,NaN,IA,NaN,...,NaN,NaN,no,75,68,0,1.0,0.0,14.0,0.0
10623,99992350,1,2,0,0,3,2.0,0.0,IIA,IIIA,...,1.0,NaN,yes,85,79,0,0.0,0.0,16.0,0.0


In [426]:
len(tumor_patients.ehr.unique())

9484

Nuestro objetivo consiste en predecir si en un paciente habra o no una recaida. No contamos con una variable directamente que nos indique si un paciente recayó o no, pero haciendo uso de este dataset podemos crear una variable asi. 

Un paciente ha recaido si ha tenido mas de un tumor, por lo que podemos construir un nuevo dataframe con una columna binaria 'recaida' si existen 2 filas con mismo 'ehr', ya que cada una correspondera a un 'n_tumor' distinto. Para predecir si un paciente recaera o no tan solo necesitamos saber si un paciente ha tenido más de un tumor, por tanto eliminaremos las filas que hacen referencia a los tumores reincidentes y de esta forma habra una unica fila por paciente para poder expresar si el paciente ha recaido o no.

In [427]:
patients = tumor_patients.copy()

# Crear la nueva variable 'recaida'
patients['recaida'] = patients['n_tumor'].apply(lambda x: 1 if x == 2 else 0)

# Asignar 'recaida' a 1 incluso en el primer valor del paciente
patients.loc[patients.duplicated(subset=['ehr'], keep='first') | ~patients.duplicated(subset=['ehr'], keep=False), 'recaida'] = 1

# Eliminar las filas duplicadas, conservando la primera aparición y la nueva variable 'recaida'
patients = patients.drop_duplicates(subset=['ehr'], keep='first')

patients.drop(columns=['n_tumor'], inplace=True)


Verificamos que efectivamente no haya IDs repetidos.

In [428]:
len(patients.ehr.unique()) == len(patients)

True

In [429]:
porcentaje_0 = (patients['recaida'] == 0).mean() * 100
porcentaje_1 = (patients['recaida'] == 1).mean() * 100

# Imprime los resultados
print(f'Porcentaje de pacientes con recaida: {porcentaje_0:.2f}%')
print(f'Porcentaje de pacientes sin recaida: {porcentaje_1:.2f}%')

Porcentaje de pacientes con recaida: 12.03%
Porcentaje de pacientes sin recaida: 87.97%


In [430]:
patients

,ehr,t_category,n_category,m_category,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_diagnosis,stage_after_neo,grade,...,lobular,neoadjuvant,age,diagnosis_age,dead,er,her2,ki67,pr,recaida
0,10011773,IS,0,0,NaN,NaN,NaN,0,NaN,1,...,NaN,no,65,56,0,0.0,0.0,19.0,NaN,1
1,10020495,1,0,0,NaN,NaN,NaN,IA,NaN,2,...,NaN,no,71,64,0,1.0,0.0,9.0,0.0,0
3,10030299,1,0,0,NaN,NaN,NaN,IA,NaN,1,...,NaN,no,58,53,0,1.0,NaN,18.0,1.0,1
4,10030824,2,1,0,2,2.0,0.0,IIIA,IIIA,3,...,1.0,yes,71,65,0,1.0,0.0,NaN,1.0,1
5,10041592,1,1,0,4,0.0,0.0,IB,IA,2,...,NaN,yes,65,59,0,0.0,0.0,65.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10620,99963879,2,3,0,3,1.0,0.0,IIIC,0,2,...,NaN,yes,65,62,0,1.0,0.0,18.0,1.0,1
10621,99966287,1,1,0,1,0.0,0.0,IB,IIA,3,...,NaN,yes,46,40,0,1.0,0.0,NaN,0.0,1
10622,99981192,1,0,0,NaN,NaN,NaN,IA,NaN,2,...,NaN,no,75,68,0,1.0,0.0,14.0,0.0,1
10623,99992350,2,0,0,3,2.0,0.0,IIA,IIIA,2,...,NaN,yes,85,79,0,0.0,0.0,16.0,0.0,1


In [431]:
patients.describe()


,ehr,n_category,m_category,n_category_after_neoadj,m_category_after_neoadj,grade,ductal,lobular,age,diagnosis_age,dead,er,her2,ki67,pr,recaida
count,9.484000e+03,9484.000000,9484.000000,5171.000000,5171.000000,9484.000000,5383.0,1312.0,9484.000000,9484.000000,9484.000000,8786.000000,8728.000000,8767.000000,8760.000000,9484.000000
mean,5.482226e+07,0.847111,0.098587,0.785728,0.099401,1.939055,1.0,1.0,63.947491,57.089519,0.112611,0.664352,0.158685,21.080301,0.666667,0.879692
std,2.604843e+07,0.988722,0.298123,0.970013,0.299228,0.697291,0.0,0.0,10.097102,10.157803,0.316133,0.472243,0.365403,21.871781,0.471431,0.325338
min,1.001177e+07,0.000000,0.000000,0.000000,0.000000,1.000000,1.0,1.0,26.000000,14.000000,0.000000,0.000000,0.000000,-57.000000,0.000000,0.000000
25%,3.209111e+07,0.000000,0.000000,0.000000,0.000000,1.000000,1.0,1.0,57.000000,50.000000,0.000000,0.000000,0.000000,12.000000,0.000000,1.000000
50%,5.465020e+07,1.000000,0.000000,0.000000,0.000000,2.000000,1.0,1.0,64.000000,57.000000,0.000000,1.000000,0.000000,18.000000,1.000000,1.000000
75%,7.739427e+07,1.000000,0.000000,1.000000,0.000000,2.000000,1.0,1.0,71.000000,64.000000,0.000000,1.000000,0.000000,27.000000,1.000000,1.000000
max,9.999442e+07,3.000000,1.000000,3.000000,1.000000,3.000000,1.0,1.0,100.000000,97.000000,1.000000,1.000000,1.000000,100.000000,1.000000,1.000000


Como podemos ver hay una variable 'lobular' para la cual faltan el 86% de los datos, por lo que en primera instancia optamos por prescindirs de esta variable. Tambien faltan los valores de 't_category_after_neoadj', 'n_category_after_neoadj', 'm_category_after_neoadj' y 'stage_after_neo' para el 45.4% de los datos. Esto es debido a que estas variables solo toman valores en pacientes a los que se les aplico un tratamiento neoadyuvante, por lo que los que no lo recibieron tienen nulo en todas estas columnas. 

Sabiendo esto podemos optar por no tener en cuenta estas variables, o crear una nueva variable que indique si el paciente recibio o no el tratamiento neoadyuvante.

In [432]:
patients.isnull().sum()

ehr                           0
t_category                    0
n_category                    0
m_category                    0
t_category_after_neoadj    4313
n_category_after_neoadj    4313
m_category_after_neoadj    4313
stage_diagnosis               0
stage_after_neo            4313
grade                         0
ductal                     4101
lobular                    8172
neoadjuvant                   0
age                           0
diagnosis_age                 0
dead                          0
er                          698
her2                        756
ki67                        717
pr                          724
recaida                       0
dtype: int64

En un primer momento parecia que faltaban una gran cantidad de datos para la variable binaria 'lobular', sin embargo si nos fijamos en sus valores todo ellos corresponden al valor '1', por lo que parece que los nulos no lo son realmente, sino que corresponderian a la clase '0'

In [433]:
print(patients.lobular.unique())
patients['lobular'] = patients['lobular'].notnull().astype(int)

[nan  1.]


Sucede lo mismo para la clase 'ductal'

In [434]:
print(patients.ductal.unique())
patients['ductal'] = patients['ductal'].notnull().astype(int)

[nan  1.]


En cuanto a las variables 't_category_after_neoadj', 'n_category_after_neoadj', 'm_category_after_neoadj' en primer lugar optamos por no considerarlas.

In [435]:
patients[patients[['t_category_after_neoadj', 'n_category_after_neoadj', 'm_category_after_neoadj', 'stage_after_neo']].isnull().all(axis=1)]


,ehr,t_category,n_category,m_category,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_diagnosis,stage_after_neo,grade,...,lobular,neoadjuvant,age,diagnosis_age,dead,er,her2,ki67,pr,recaida
0,10011773,IS,0,0,NaN,NaN,NaN,0,NaN,1,...,0,no,65,56,0,0.0,0.0,19.0,NaN,1
1,10020495,1,0,0,NaN,NaN,NaN,IA,NaN,2,...,0,no,71,64,0,1.0,0.0,9.0,0.0,0
3,10030299,1,0,0,NaN,NaN,NaN,IA,NaN,1,...,0,no,58,53,0,1.0,NaN,18.0,1.0,1
6,10053435,0,0,0,NaN,NaN,NaN,IV,NaN,3,...,0,no,67,60,0,1.0,0.0,NaN,1.0,1
12,10115313,1,0,1,NaN,NaN,NaN,IA,NaN,3,...,0,no,67,62,0,1.0,0.0,14.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10612,99859482,0,3,0,NaN,NaN,NaN,IV,NaN,2,...,0,no,60,58,0,NaN,0.0,19.0,1.0,1
10616,99915606,IS,0,0,NaN,NaN,NaN,IIB,NaN,2,...,0,no,63,56,0,0.0,0.0,49.0,1.0,1
10617,99948515,1,0,0,NaN,NaN,NaN,IA,NaN,3,...,1,no,70,64,0,0.0,0.0,25.0,1.0,1
10618,99948591,2,0,0,NaN,NaN,NaN,IIA,NaN,2,...,1,no,69,67,0,1.0,0.0,12.0,1.0,1


Procedemos a eliminar las columnas discutidas.

In [436]:
patients.drop(columns=['t_category_after_neoadj', 'n_category_after_neoadj', 'm_category_after_neoadj','stage_after_neo'], inplace=True)


In [437]:
patients

,ehr,t_category,n_category,m_category,stage_diagnosis,grade,ductal,lobular,neoadjuvant,age,diagnosis_age,dead,er,her2,ki67,pr,recaida
0,10011773,IS,0,0,0,1,0,0,no,65,56,0,0.0,0.0,19.0,NaN,1
1,10020495,1,0,0,IA,2,1,0,no,71,64,0,1.0,0.0,9.0,0.0,0
3,10030299,1,0,0,IA,1,1,0,no,58,53,0,1.0,NaN,18.0,1.0,1
4,10030824,2,1,0,IIIA,3,0,1,yes,71,65,0,1.0,0.0,NaN,1.0,1
5,10041592,1,1,0,IB,2,1,0,yes,65,59,0,0.0,0.0,65.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10620,99963879,2,3,0,IIIC,2,1,0,yes,65,62,0,1.0,0.0,18.0,1.0,1
10621,99966287,1,1,0,IB,3,1,0,yes,46,40,0,1.0,0.0,NaN,0.0,1
10622,99981192,1,0,0,IA,2,0,0,no,75,68,0,1.0,0.0,14.0,0.0,1
10623,99992350,2,0,0,IIA,2,1,0,yes,85,79,0,0.0,0.0,16.0,0.0,1


Como podemos observar las variables 't_category', 'stage_diagnosis' y 'neoadjuvant' son categoricas, por lo que aplicaremos OneHotEncoder para que adopten valores numericos. Para 'neoadjuvant' simplemente daremos el valor '0' a 'no' y '1' a 'yes'.

In [438]:
from sklearn.preprocessing import OneHotEncoder

patients['neoadjuvant'] = patients['neoadjuvant'].map({'no': 0, 'yes': 1})

encoder = OneHotEncoder(sparse=False)
columns_to_encode = ['t_category', 'stage_diagnosis']

encoded_columns = encoder.fit_transform(patients[columns_to_encode])
# encoded_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(columns_to_encode))


# t_catOHE = encoder.fit_transform(patients[['t_category']])
# t_cols = encoder.get_feature_names_out(['t_category'])
patients[encoder.get_feature_names_out(columns_to_encode)] = encoded_columns
patients.drop(columns=['t_category', 'stage_diagnosis'], inplace=True)
patients

,ehr,n_category,m_category,grade,ductal,lobular,neoadjuvant,age,diagnosis_age,dead,...,t_category_IS,stage_diagnosis_0,stage_diagnosis_IA,stage_diagnosis_IB,stage_diagnosis_IIA,stage_diagnosis_IIB,stage_diagnosis_IIIA,stage_diagnosis_IIIB,stage_diagnosis_IIIC,stage_diagnosis_IV
0,10011773,0,0,1,0,0,0,65,56,0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10020495,0,0,2,1,0,0,71,64,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10030299,0,0,1,1,0,0,58,53,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10030824,1,0,3,0,1,1,71,65,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,10041592,1,0,2,1,0,1,65,59,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10620,99963879,3,0,2,1,0,1,65,62,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10621,99966287,1,0,3,1,0,1,46,40,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10622,99981192,0,0,2,0,0,0,75,68,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10623,99992350,0,0,2,1,0,1,85,79,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [439]:
patients.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9484 entries, 0 to 10624
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ehr                   9484 non-null   int64  
 1   n_category            9484 non-null   int64  
 2   m_category            9484 non-null   int64  
 3   grade                 9484 non-null   int64  
 4   ductal                9484 non-null   int32  
 5   lobular               9484 non-null   int32  
 6   neoadjuvant           9484 non-null   int64  
 7   age                   9484 non-null   int64  
 8   diagnosis_age         9484 non-null   int64  
 9   dead                  9484 non-null   int32  
 10  er                    8786 non-null   float64
 11  her2                  8728 non-null   float64
 12  ki67                  8767 non-null   float64
 13  pr                    8760 non-null   float64
 14  recaida               9484 non-null   int64  
 15  t_category_0        

In [440]:
patients.isnull().sum()

ehr                       0
n_category                0
m_category                0
grade                     0
ductal                    0
lobular                   0
neoadjuvant               0
age                       0
diagnosis_age             0
dead                      0
er                      698
her2                    756
ki67                    717
pr                      724
recaida                   0
t_category_0              0
t_category_1              0
t_category_2              0
t_category_3              0
t_category_4              0
t_category_IS             0
stage_diagnosis_0         0
stage_diagnosis_IA        0
stage_diagnosis_IB        0
stage_diagnosis_IIA       0
stage_diagnosis_IIB       0
stage_diagnosis_IIIA      0
stage_diagnosis_IIIB      0
stage_diagnosis_IIIC      0
stage_diagnosis_IV        0
dtype: int64

Una vez tenemos este dataframe procedemos a crear otro que disponga tambien de la informacion ginecologica.


In [441]:
patients_gyn = pd.merge(patients, gynecological_patients, on='ehr')

Como se puede apreciar este es un dataset con menos pacientes (2318 paacientes menos) pero dispone de mas variables predictoras, por lo que puede brindar mejores resultados. Aun asi estas hipotesis deberan ser verificadas experimentalmente.

In [442]:
patients_gyn

,ehr,n_category,m_category,grade,ductal,lobular,neoadjuvant,age_x,diagnosis_age_x,dead_x,...,caesarean,abort,menarche_age,age_y,diagnosis_age_y,dead_y,er_y,her2_y,ki67_y,pr_y
0,10011773,0,0,1,0,0,0,65,56,0,...,NaN,0.0,NaN,65,56,0,0.0,0.0,19.0,NaN
1,10030299,0,0,1,1,0,0,58,53,0,...,0.0,0.0,NaN,58,53,0,1.0,NaN,18.0,1.0
2,10030824,1,0,3,0,1,1,71,65,0,...,0.0,NaN,NaN,71,65,0,1.0,0.0,NaN,1.0
3,10053435,0,0,3,1,0,0,67,60,0,...,0.0,1.0,21.0,67,60,0,1.0,0.0,NaN,1.0
4,10111454,0,0,2,1,0,1,63,56,0,...,NaN,0.0,16.0,63,56,0,1.0,0.0,72.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7161,99948591,0,0,2,0,1,0,69,67,0,...,NaN,0.0,12.0,69,67,0,1.0,0.0,12.0,1.0
7162,99961100,2,0,3,1,0,1,54,48,0,...,0.0,0.0,19.0,54,48,0,1.0,0.0,21.0,1.0
7163,99981192,0,0,2,0,0,0,75,68,0,...,0.0,NaN,11.0,75,68,0,1.0,0.0,14.0,0.0
7164,99992350,0,0,2,1,0,1,85,79,0,...,NaN,NaN,11.0,85,79,0,0.0,0.0,16.0,0.0


## IMPUTACION DE NULOS PATIENTS

Como podemos comprobar para cada fila, como maximo, hay un unico valor nulo. Ante esto se nos plantean 2 posibilidades:

* Imputar los valores (KNNImputer)
* Prescindir de estas filas (Nos quedariamos con 9485 - 2878 = 6607 datos.)

In [443]:
filas_nulas = patients[patients.columns.drop('ehr')].isnull().sum(axis=1) >=2
len(filas_nulas[filas_nulas].index.tolist())

0

En una primera aproximacion optaremos por imputar los valores mediante KNNImputer

In [444]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2)
patients =pd.DataFrame(imputer.fit_transform(patients), columns=patients.columns)

In [445]:
patients

,ehr,n_category,m_category,grade,ductal,lobular,neoadjuvant,age,diagnosis_age,dead,...,t_category_IS,stage_diagnosis_0,stage_diagnosis_IA,stage_diagnosis_IB,stage_diagnosis_IIA,stage_diagnosis_IIB,stage_diagnosis_IIIA,stage_diagnosis_IIIB,stage_diagnosis_IIIC,stage_diagnosis_IV
0,10011773.0,0.0,0.0,1.0,0.0,0.0,0.0,65.0,56.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10020495.0,0.0,0.0,2.0,1.0,0.0,0.0,71.0,64.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10030299.0,0.0,0.0,1.0,1.0,0.0,0.0,58.0,53.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10030824.0,1.0,0.0,3.0,0.0,1.0,1.0,71.0,65.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,10041592.0,1.0,0.0,2.0,1.0,0.0,1.0,65.0,59.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9479,99963879.0,3.0,0.0,2.0,1.0,0.0,1.0,65.0,62.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9480,99966287.0,1.0,0.0,3.0,1.0,0.0,1.0,46.0,40.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9481,99981192.0,0.0,0.0,2.0,0.0,0.0,0.0,75.0,68.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9482,99992350.0,0.0,0.0,2.0,1.0,0.0,1.0,85.0,79.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [446]:
porcentaje_0 = (patients['recaida'] == 0).mean() * 100
porcentaje_1 = (patients['recaida'] == 1).mean() * 100

# Imprime los resultados
print(f'Porcentaje de pacientes con recaida: {porcentaje_0:.2f}%')
print(f'Porcentaje de pacientes sin recaida: {porcentaje_1:.2f}%')

Porcentaje de pacientes con recaida: 12.03%
Porcentaje de pacientes sin recaida: 87.97%


Verificamos que efectivamente no haya valores nulos:

In [447]:
patients.isnull().sum()

ehr                     0
n_category              0
m_category              0
grade                   0
ductal                  0
lobular                 0
neoadjuvant             0
age                     0
diagnosis_age           0
dead                    0
er                      0
her2                    0
ki67                    0
pr                      0
recaida                 0
t_category_0            0
t_category_1            0
t_category_2            0
t_category_3            0
t_category_4            0
t_category_IS           0
stage_diagnosis_0       0
stage_diagnosis_IA      0
stage_diagnosis_IB      0
stage_diagnosis_IIA     0
stage_diagnosis_IIB     0
stage_diagnosis_IIIA    0
stage_diagnosis_IIIB    0
stage_diagnosis_IIIC    0
stage_diagnosis_IV      0
dtype: int64

Finalmente, creamos un daset normalizado para su uso en alguno de los modelos que usaremos. Habra otros modelos en los que no sea necesdaria la normalizacion como Decision Trees, Random Forest o Gradient Boosting en general. 

In [448]:
from sklearn.preprocessing import MinMaxScaler

#Eliminamos el id del paciente
patients.drop(columns='ehr', inplace=True)

scaler = MinMaxScaler()
patients_norm = patients.copy()

#Automatizamos el proceso de identificar variables binarias y no tenerlas en cuenta para la normalizacion.
norm_cols = [col for col in patients_norm.columns if sorted(patients_norm[col].unique()) != [0, 1]]

# No queremos normalizar las columnas binarias.
patients_norm[norm_cols] = scaler.fit_transform(patients_norm[norm_cols])


In [449]:
patients_norm

,n_category,m_category,grade,ductal,lobular,neoadjuvant,age,diagnosis_age,dead,er,...,t_category_IS,stage_diagnosis_0,stage_diagnosis_IA,stage_diagnosis_IB,stage_diagnosis_IIA,stage_diagnosis_IIB,stage_diagnosis_IIIA,stage_diagnosis_IIIB,stage_diagnosis_IIIC,stage_diagnosis_IV
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.527027,0.506024,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.5,1.0,0.0,0.0,0.608108,0.602410,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,1.0,0.0,0.0,0.432432,0.469880,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.333333,0.0,1.0,0.0,1.0,1.0,0.608108,0.614458,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.333333,0.0,0.5,1.0,0.0,1.0,0.527027,0.542169,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9479,1.000000,0.0,0.5,1.0,0.0,1.0,0.527027,0.578313,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9480,0.333333,0.0,1.0,1.0,0.0,1.0,0.270270,0.313253,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9481,0.000000,0.0,0.5,0.0,0.0,0.0,0.662162,0.650602,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9482,0.000000,0.0,0.5,1.0,0.0,1.0,0.797297,0.783133,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [450]:
patients.to_csv('recaida.csv')
patients_norm.to_csv('recaida_norm.csv')



## IMPUTACION DE NULOS PATIENS_GYN

In [451]:
filas_nulas = patients_gyn[patients_gyn.columns.drop('ehr')].isnull().sum(axis=1) >=5
len(filas_nulas[filas_nulas].index.tolist())

233

In [452]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2)
patients_gyn =pd.DataFrame(imputer.fit_transform(patients_gyn), columns=patients_gyn.columns)

In [453]:
patients_gyn

,ehr,n_category,m_category,grade,ductal,lobular,neoadjuvant,age_x,diagnosis_age_x,dead_x,...,caesarean,abort,menarche_age,age_y,diagnosis_age_y,dead_y,er_y,her2_y,ki67_y,pr_y
0,10011773.0,0.0,0.0,1.0,0.0,0.0,0.0,65.0,56.0,0.0,...,0.0,0.0,18.5,65.0,56.0,0.0,0.0,0.0,19.0,1.0
1,10030299.0,0.0,0.0,1.0,1.0,0.0,0.0,58.0,53.0,0.0,...,0.0,0.0,18.5,58.0,53.0,0.0,1.0,0.0,18.0,1.0
2,10030824.0,1.0,0.0,3.0,0.0,1.0,1.0,71.0,65.0,0.0,...,0.0,0.0,18.5,71.0,65.0,0.0,1.0,0.0,18.5,1.0
3,10053435.0,0.0,0.0,3.0,1.0,0.0,0.0,67.0,60.0,0.0,...,0.0,1.0,21.0,67.0,60.0,0.0,1.0,0.0,18.5,1.0
4,10111454.0,0.0,0.0,2.0,1.0,0.0,1.0,63.0,56.0,0.0,...,0.0,0.0,16.0,63.0,56.0,0.0,1.0,0.0,72.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7161,99948591.0,0.0,0.0,2.0,0.0,1.0,0.0,69.0,67.0,0.0,...,0.0,0.0,12.0,69.0,67.0,0.0,1.0,0.0,12.0,1.0
7162,99961100.0,2.0,0.0,3.0,1.0,0.0,1.0,54.0,48.0,0.0,...,0.0,0.0,19.0,54.0,48.0,0.0,1.0,0.0,21.0,1.0
7163,99981192.0,0.0,0.0,2.0,0.0,0.0,0.0,75.0,68.0,0.0,...,0.0,0.0,11.0,75.0,68.0,0.0,1.0,0.0,14.0,0.0
7164,99992350.0,0.0,0.0,2.0,1.0,0.0,1.0,85.0,79.0,0.0,...,0.0,0.0,11.0,85.0,79.0,0.0,0.0,0.0,16.0,0.0


In [454]:
porcentaje_0 = (patients_gyn['recaida'] == 0).mean() * 100
porcentaje_1 = (patients_gyn['recaida'] == 1).mean() * 100

# Imprime los resultados
print(f'Porcentaje de pacientes con recaida: {porcentaje_0:.2f}%')
print(f'Porcentaje de pacientes sin recaida: {porcentaje_1:.2f}%')

Porcentaje de pacientes con recaida: 12.36%
Porcentaje de pacientes sin recaida: 87.64%


In [455]:
patients_gyn.isnull().sum()

ehr                     0
n_category              0
m_category              0
grade                   0
ductal                  0
lobular                 0
neoadjuvant             0
age_x                   0
diagnosis_age_x         0
dead_x                  0
er_x                    0
her2_x                  0
ki67_x                  0
pr_x                    0
recaida                 0
t_category_0            0
t_category_1            0
t_category_2            0
t_category_3            0
t_category_4            0
t_category_IS           0
stage_diagnosis_0       0
stage_diagnosis_IA      0
stage_diagnosis_IB      0
stage_diagnosis_IIA     0
stage_diagnosis_IIB     0
stage_diagnosis_IIIA    0
stage_diagnosis_IIIB    0
stage_diagnosis_IIIC    0
stage_diagnosis_IV      0
pregnancy               0
birth                   0
caesarean               0
abort                   0
menarche_age            0
age_y                   0
diagnosis_age_y         0
dead_y                  0
er_y        

In [456]:
from sklearn.preprocessing import MinMaxScaler

#Eliminamos el id del paciente
patients_gyn.drop(columns='ehr', inplace=True)

scaler = MinMaxScaler()
patients_gyn_norm = patients_gyn.copy()

#Automatizamos el proceso de identificar variables binarias y no tenerlas en cuenta para la normalizacion.
norm_cols = [col for col in patients_gyn_norm.columns if sorted(patients_gyn_norm[col].unique()) != [0, 1]]

# No queremos normalizar las columnas binarias.
patients_gyn_norm[norm_cols] = scaler.fit_transform(patients_gyn_norm[norm_cols])


In [457]:
patients_gyn_norm

,n_category,m_category,grade,ductal,lobular,neoadjuvant,age_x,diagnosis_age_x,dead_x,er_x,...,caesarean,abort,menarche_age,age_y,diagnosis_age_y,dead_y,er_y,her2_y,ki67_y,pr_y
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.527027,0.506024,0.0,0.0,...,0.0,0.000000,0.711538,0.527027,0.506024,0.0,0.0,0.0,0.484076,1.0
1,0.000000,0.0,0.0,1.0,0.0,0.0,0.432432,0.469880,0.0,1.0,...,0.0,0.000000,0.711538,0.432432,0.469880,0.0,1.0,0.0,0.477707,1.0
2,0.333333,0.0,1.0,0.0,1.0,1.0,0.608108,0.614458,0.0,1.0,...,0.0,0.000000,0.711538,0.608108,0.614458,0.0,1.0,0.0,0.480892,1.0
3,0.000000,0.0,1.0,1.0,0.0,0.0,0.554054,0.554217,0.0,1.0,...,0.0,0.083333,0.807692,0.554054,0.554217,0.0,1.0,0.0,0.480892,1.0
4,0.000000,0.0,0.5,1.0,0.0,1.0,0.500000,0.506024,0.0,1.0,...,0.0,0.000000,0.615385,0.500000,0.506024,0.0,1.0,0.0,0.821656,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7161,0.000000,0.0,0.5,0.0,1.0,0.0,0.581081,0.638554,0.0,1.0,...,0.0,0.000000,0.461538,0.581081,0.638554,0.0,1.0,0.0,0.439490,1.0
7162,0.666667,0.0,1.0,1.0,0.0,1.0,0.378378,0.409639,0.0,1.0,...,0.0,0.000000,0.730769,0.378378,0.409639,0.0,1.0,0.0,0.496815,1.0
7163,0.000000,0.0,0.5,0.0,0.0,0.0,0.662162,0.650602,0.0,1.0,...,0.0,0.000000,0.423077,0.662162,0.650602,0.0,1.0,0.0,0.452229,0.0
7164,0.000000,0.0,0.5,1.0,0.0,1.0,0.797297,0.783133,0.0,0.0,...,0.0,0.000000,0.423077,0.797297,0.783133,0.0,0.0,0.0,0.464968,0.0


In [458]:
patients_gyn.to_csv('recaida_gyn.csv')
patients_gyn_norm.to_csv('recaida_gyn_norm.csv')

